In [ ]:
# import glob
# import os
# from bs4 import BeautifulSoup

# folder = 'Telegraaf'
# document_top = 1

# for infile in glob.glob( os.path.join(folder, '*.xml'))[:document_top]:
#         # get the html text
#         fileIndex = os.path.basename(infile).replace('.xml','')
#         print("Current file is {}".format(fileIndex))

#         print("\tReading...")
#         # Open the file and read it
#         with open(infile, 'r', encoding='utf-8') as f:
#             soup = BeautifulSoup(f.read(), "html.parser")
            
#             print("\tFormatting...")
#             all_roots = soup.findAll("pm:root".split())
            
#             print("\tSaving...")
#             for root in all_roots:
#                 print(root.prettify())

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch()

# ignore 400 cause by IndexAlreadyExistsException when creating an index
es.indices.create(index='test-index', ignore=400)

# ignore 404 and 400
es.indices.delete(index='test-index', ignore=[400, 404])

In [ ]:
import glob
import os
from bs4 import BeautifulSoup
import sys
import gzip

folder = 'Telegraaf'

total_amount_records = 0
for infile in glob.glob( os.path.join(folder, '*.xml.gz')):
        # get the html text
        fileIndex = os.path.basename(infile).replace('.xml.gz','')
        print("")
        print("Current file is {}".format(fileIndex))

        print("\tOpening...")
        # Open the file and read it
        with gzip.open(infile, 'r') as f:
            soup = BeautifulSoup(f.read(), "html.parser")
            
            print("\tFormatting...")
            all_roots = soup.findAll("pm:root".split())
            total_amount_roots = len(all_roots)
            current_index_root = 0
            
            print("\tReading and saving...")
            for root in all_roots:
                
                _date = root.find("dc:date").text
                subject = root.find("dc:subject").text
                identifier = root.find("dc:identifier").text
#                 link = root.find("pm:link").text # de source is dan http://kranten.kb.nl/view/article/id/ + de link? <KLOPT>
#                 source_denk_ik = "http://kranten.kb.nl/view/article/id/{}".format(identifier)
                title = root.find("title").text
                text = root.find("text").text
                
#                 total_p = ""
#                 for p in root.find("text").findAll("p"):
#                     total_p = "{}\n\t{}".format(total_p,p.text)
                    
#                 print('\ndate:{},  subject:{}  \nidentifier: {}, link: {}, source: {} \ntitle: {}, text: {} \ntotal_p: {}\n\n'.format(
#                     _date,subject,identifier,link,source_denk_ik,title,text,total_p))
                
                
                
                es.index(index='krant',doc_type=subject, id=identifier, body={
                    'title': title,
                    'text': text,
                    'date': _date
                })
                

                
                current_index_root += 1
                sys.stdout.write('\r' + "\t\t{}/{}".format(current_index_root,total_amount_roots))
                sys.stdout.flush()
        total_amount_records += total_amount_roots
        
print("Everything is done, a total of {} records are saved".format(total_amount_records))
                
#  1969 is af, 1970 was bezig

In [ ]:
es.search(doc_type='artikel', q='PRIVÉ WILLEKE KRIJGT HOOFDROL BIJ VERONIOUE')